<a href="https://colab.research.google.com/github/ProteinEngineer/Protein_codes/blob/main/PyRosetta_docking_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.19+release.a34b73c40fe9c61558d566d6a63f803cfb15a4fc 2024-05-02T16:22:03] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r381 2024.19+release.a34b73c40f a34b

In [15]:
from pyrosetta import *
from pyrosetta.rosetta.core.pose import Pose
from pyrosetta.rosetta.protocols.docking import DockMCMProtocol, DockingInitialPerturbation, DockingSlideIntoContact
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.protocols.moves import MonteCarlo
init()

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.19+release.a34b73c40fe9c61558d566d6a63f803cfb15a4fc 2024-05-02T16:22:03] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r381 2024.19+release.a34b73c40f a34b

In [16]:
protein_pose = pose_from_pdb("/content/3h1d.pdb")
ligand_pose = pose_from_pdb("/content/HU.1.pdb")
control_pose = pose_from_pdb("/content/5hps.pdb")
scorefxn = get_fa_scorefxn()
print(scorefxn(control_pose))

core.import_pose.import_pose: File '/content/3h1d.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 382
core.import_pose.import_pose: File '/content/HU.1.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 1 in file /content/HU.1.pdb. Best match rsd_type:  MET:NtermProteinFull
core.import_pose.import_pose: File '/content/5hps.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 28
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NZ  on residue LYS 28
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 374
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 447
core.pack.pack_missing_sidechains: packing residue number 28 beca

In [17]:
complex_pose = protein_pose.clone()

complex_pose.append_pose_by_jump(ligand_pose, complex_pose.total_residue())
# Initialize a new FoldTree
fold_tree = pyrosetta.rosetta.core.kinematics.FoldTree()

# Number of residues in the receptor and ligand
n_res_receptor = protein_pose.total_residue()
n_res_ligand = ligand_pose.total_residue()

# Set up the FoldTree
fold_tree.add_edge(1, n_res_receptor, -1)  # Peptide bond within the receptor
fold_tree.add_edge(1, n_res_receptor + 1, 1)  # Jump from receptor to ligand
fold_tree.add_edge(n_res_receptor + 1, n_res_receptor + n_res_ligand, -1)  # Peptide bond within the ligand

# Apply the FoldTree to the combined pose
complex_pose.fold_tree(fold_tree)
'''
# Set the fold tree for the complex pose
docking_ft = pyrosetta.rosetta.core.kinematics.FoldTree()
docking_ft.add_edge(1, _pose.total_residue(), -1)
docking_ft.add_edge(1, complex_pose.total_residue(), 1)
complex_pose.fold_tree(docking_ft)
'''
scorefxn = pyrosetta.get_fa_scorefxn()

# Optionally perform low-resolution docking first
dock_prot = DockMCMProtocol()
dock_prot.apply(complex_pose)

#from pyrosetta.rosetta.protocols.docking import DockingHighResProtocol
'''
# Create a high-resolution docking protocol
high_res_dock = pyrosetta.rosetta.protocols.HighResDocking()
high_res_dock.set_scorefxn(scorefxn)
high_res_dock.set_protein_pose(protein_pose)
high_res_dock.set_ligand_pose(ligand_pose)

# Apply the high-resolution docking protocol to the combined pose
high_res_dock.apply(complex_pose)
'''

from pyrosetta.rosetta.protocols.minimization_packing import MinMover

# Create a MoveMap to allow backbone and side-chain movements
movemap = MoveMap()
movemap.set_bb(True)
movemap.set_chi(True)

# Create a MinMover for further minimization
min_mover = MinMover()
min_mover.movemap(movemap)
min_mover.score_function(scorefxn)
min_mover.min_type('lbfgs_armijo_nonmonotone')  # Use a robust minimization algorithm

# Apply the minimization to the combined pose
min_mover.apply(complex_pose)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
protocols.docking.DockMCMProtocol: in DockMCMProtocol.apply
protocols.docking.DockMCMProtocol: Using the DockingTaskFactory.
protocols.docking.DockMCMCycle: Setting up defaults for DockMCMCycle:
protocols.docking.DockMCMCycle: ////////////////////////////////////////////////////////////////////////////////
protocols.docking.DockMCMCycle: /// Dockable Jumps: 1                                                        ///
protocols.docking.DockMCMCycle: /// Translation: 0.1                                                         ///
protocols.docking.DockMCMCycle: /// Rotation: 5                                                              ///
protocols.docking.DockMCMCycle: /// Scorefunction:                                                           ///
protocol

In [19]:
# Get the scoring function
scorefxn = pyrosetta.get_fa_scorefxn()

# Score the docked complex
complex_score = scorefxn(complex_pose)
print("Docked Complex Score:", complex_score)

# Score the protein alone
protein_score = scorefxn(protein_pose)
print("Protein Score:", protein_score)

# Score the ligand alone
ligand_score = scorefxn(ligand_pose)
print("Ligand Score:", ligand_score)

# Calculate binding energy
binding_energy = complex_score - (protein_score + ligand_score)
print("Binding Energy:", binding_energy)

# Output the docked complex structure to a PDB file
complex_pose.dump_pdb("docked_complex.pdb")

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
Docked Complex Score: -1266.4980860198093
Protein Score: -227.69940413181143
Ligand Score: -103.17755104770693
Binding Energy: -935.621130840291


True